In [1]:
using Images, FileIO, PyPlot

# Parameters we can learn
#     Gaussian comparison
#         Spread
         
#     Center^p minus (average of corners)^q
#         p
#         q
    
#     Vetting the kernel
#         Threshold for sum of entire kernel
#         Other methods with other parameters?
            
    
#     Kernel size
#         5, 7 or 9
    
#     Final threshold
#         t
    
# Other things to think about
#     Normalize the image between 0-1 or normalize the kernel between 0-1? Would this make a difference? 
#     Combine methods? For example, standard deviation + center minus corners?

#     Three dimensionality
#         Should we do the dice function on the stack (voxel) level or on the slice (pixel) level? 
#         Could we do the slice level and then somehow use this to create, train and execute the slice level?

#     Training data
#         Would it be worth it to hand-annote not just the center of each focus, but each individual pixel? 
#         Or should the method be robust enough in the first place to not need that?



In [2]:
function ddfload(filename)
    # Loads the entire stack of 7 and returns a normalized (0-1) Array{Float64}
    img = load(filename)
    img = Float64.(img)
    img = img .- minimum(img)
    img = img./maximum(img)
    return Float64.(img)
end

function MIP(img)
    img = Float32.(img)
    img = maximum(img, 3)
    img = Float64.(img)
    return img[:, :, 1]
end


function weightedcentroid(focus)
    # Calculate the weighted centroid coordinates
    m = size(focus, 1)
    n = size(focus, 2)

    xsum = sum(focus, 1)
    ysum = sum(focus, 2)
    
    
    weightedsumx = 1*xsum[2]
    for i = 2:n
       weightedsumx = weightedsumx + i*xsum[i] # tack on i*xsum[i]
    end
    
    Cx = weightedsumx/sum(xsum)
    
    weightedsumy = 1*ysum[2]
    
    for i = 2:n
       weightedsumy = weightedsumy + i*ysum[i]
    end
    
    Cy = weightedsumy/sum(ysum)
    
    return (Cx, Cy)
    
end


function generateidealgaussian(m, n, spread, z, Cx, Cy)
    sigmax = spread
    sigmay = spread
    m = z*m
    n = z*n
    x0 = z*Cx
    y0 = z*Cy

    gauss = zeros(m, n)
    
    for x = 1:m

        for y = 1:n
            a = ((x-x0)^2)/(2*sigmax^2)
            b = ((y-y0)^2)/(2*sigmay^2)
            gauss[x, y] = exp(-( (a) + (b) ))
        end
    end
    
    
    # Normalize
#     gauss = gauss - minimum(gauss)
#     gauss = gauss./maximum(gauss)
    return Float64.(gauss)
end

function discretizegaussian(G, m, n, z)
    # return m x n to be overlaid
    # for now use averaging
    
    Gdiscrete = zeros(m, n)
    
    for x = 1:m
        for y = 1:n
            
            
            
            mstart = 1 + z*(x-1)
            mend = x*z
            nstart = 1 + z*(y-1)
            nend = y*z

            chunk = G[mstart:mend, nstart:nend]
            if x == 3 & y == 1
               singlechunk = chunk
            end
            
            Gdiscrete[x, y] = sum(chunk) / (length(chunk))
            
        end
    end

    return Gdiscrete
end

function vetkernel(kernel, thresh)

    if sum(kernel) < thresh # If it didn't make the threshold
        return false
    else # If it did, get the weighted centroid then make the new box
        return true
    end
    
    return true

end

# Try out different score heuristics here

function comparetogaussian(kernel, Cx, Cy, z, spread)
    # Generate a m x n gaussian 
    spread = z*2
    score = 0
   
    if kernel != false && Cx != false
        (m, n) = size(kernel)
                
        G = generateidealgaussian(m, n, spread, z, Cx, Cy)
        G = G 
        # Make amplitude equal to max of focus
        Gdiscrete = discretizegaussian(G, m, n, z)
        if size(Gdiscrete) != size(kernel)
          score = 0 
        else
            score = sum((Gdiscrete .- kernel)^2)
        end

    end 
    
    return score
end

function stdscore(newfocus)
    if newfocus != false
        return std(vec(newfocus))
    else
        return 0
    end
end

function centerminuscorner(kernel)
    if kernel != false
        return kernel[3, 3] - (1/4)*(kernel[1, 1] + kernel[size(kernel, 1), 1] + kernel[1, size(kernel, 2)] + kernel[size(kernel, 1), size(kernel, 2)])
    else
        return 0
    end
end

function centerovercorner(newfocus)
    if newfocus != false
        avgcorners = (1/4)*(newfocus[1, 1] + newfocus[size(newfocus, 1), 1] + newfocus[1, size(newfocus, 2)] + newfocus[size(newfocus, 1), size(newfocus, 2)])
        return (newfocus[3, 3]) / avgcorners
    else
        return 0
    end
end

function normalize01(img)
    img = img - minimum(img)
    img = img./maximum(img)
    return Float64.(img)
end

function plot3things(A, B, C)
    fig = figure("pyplot_subplot_mixed",figsize=(18, 10)) # Create a new blank figure
    subplot(131)
    title("Thing A")
    PyPlot.imshow(A)
    
    subplot(132) # Create the second plot of a 3x1 group of subplots
    title("Thing B")
    PyPlot.imshow(B)
    
    subplot(133)
    title("Thing C")
    PyPlot.imshow(C)
end

plot3things (generic function with 1 method)

In [3]:
function segmentslice(img, n, threshold)
    z = 10 # resolution of gaussian
    r = Int16.((n-1)/2)
    
    poss = 0
    count = 0
    
    thresh = 1.5
    
    # Have to start at [3, 3] for a 5x5 kernel 
    xstart = round((n/2)+1)
    xend = size(img, 1) - xstart
    ystart = round((n/2)+1)
    yend = size(img, 2) - ystart
    xstart = Int16.(xstart)
    xend = Int16.(xend)
    ystart = Int16.(ystart)
    yend = Int16.(yend)


    scores = zeros(size(img))
    stdev = zeros(size(img))
    spread = 3
    
    for x = xstart:xend
        for y = ystart:yend
            kernel = img[x-r:x+r,y-r:y+r]
            if vetkernel(kernel, 1.5) == false
                scores[x, y] = 0
            else
                Cx, Cy = weightedcentroid(kernel)
                scores[x, y] = centerminuscorner(kernel)
            end
            
        end
    end
    
    scores = normalize01(scores)
    bw = scores .> threshold
    
    return scores
end

segmentslice (generic function with 1 method)

In [7]:
stack = ddfload("GFP/GFP_006.tif")
slice = stack[:, :, 1]
segmented = segmentslice(slice, 7, .5).>.5
labels = label_components(segmented, trues(3, 3))
indices = component_indices(labels)

6-element Array{Array{Int64,1},1}:
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  262135, 262136, 262137, 262138, 262139, 262140, 262141, 262142, 262143, 262144]                             
 [46847, 46848, 46849, 47361]                                                                                                                   
 [46851, 46852, 46853, 47363, 47364, 47365, 47876, 47877]                                                                                       
 [79674, 79675, 79676, 79677, 80186, 80187, 80188, 80189, 80697, 80698  …  80700, 80701, 81209, 81210, 81211, 81212, 81213, 81722, 81723, 81724]
 [135251, 135252, 135762, 135763, 135764, 135765, 136274, 136275, 136276, 136277, 136786, 136787, 136788, 136789, 137299, 137300]               
 [207567, 207568, 208079, 208080, 208592]                                                                                                       

In [8]:
@show length(indices[1])

length(indices[1]) = 262090


262090